
<div align="center">

## Music Composition Using Quantum Annealing
### Ashish Arya, Ludmila Botelho, Fabiola Canete, Dhruvi Kapadia, Özlem Salehi	
</div>


This notebook contains the codes used for generating the music pieces presented in the chapter "Music Composition Using Quantum Annealing". It is prepared to guide you while reading the chapter.

Person responsible for the codes: Özlem Salehi (osalehi@iitis.pl)

---

### Installation

In order to run the codes, you need to install the following packages:
- Ocean Software
- pickle

To install Ocean Software, you may use the command `pip install dwave-ocean-sdk`. You can visit [Ocean](https://docs.ocean.dwavesys.com/en/stable/overview/install.html) webpage for further information. 

After the installation, follow the guidelines in the Ocean webpage to configure your system. Depending on your country of residence, you may not be able to access D-Wave machines. Nevertheless, you can still run the codes using simulated annealer.

We will use the [pickle](https://docs.python.org/3.8/library/pickle.html) module to save data. You can install it by the command `pip install pickle` or omit it if you don't want to save the outputs.


### Imports

You need to import the following to run the codes. If you have made a successfull installation, you should be able to run the following cell without any error. 

In [48]:
print("Hello world!")

Hello world!


In [49]:
# D-Wave modules
import dimod
import neal
from dwave.system.samplers.leap_hybrid_sampler import LeapHybridSampler
from dwave.system import EmbeddingComposite, DWaveSampler
import dwave_networkx as dnx 

# Pyqubo will be used formulate the models
from pyqubo import Binary, Constraint, LogEncInteger

# pickle will be used for saving data
import pickle

# This is a built in module which we will use in the header of the for loops.
from itertools import product

# MIDI player
import midi
# For alphabet
import string

### Helper functions 

We will list some functions that will be used in the rest of the notebook.

<b>Storing results</b>

The sampleset returned by D-Wave is converted in to a serializable object which is convenient for storing and pickle is used to store it.

In [50]:
def store_results(filename, sampleset):
    """Stores the sampleset obtained from D-Wave
    
    :param filename: name of the file 
    :type filename: string
    :param sampleset: sampleset to be stored
    :type sampleset: dimod.SampleSet
    """
    sdf = sampleset.to_serializable()
    with open(filename, 'wb') as handle:
        pickle.dump(sdf, handle)

<b> Annealing experiment</b>

This function is used to perform the annealing experiment. Annealing type is either 's', 'h' or 'q', representing simulated, hybrid and quantum respectively. The parameter `ns` which stands for number of sweeps is needed only for simulated annealing. `at` is the annealing time for quantum annealing and `cs` determines the chain strength. For hybrid annealing, no parameters are used. If any of the parameters are not provided, then their default values are used.

In [51]:
def anneal(bqm, anneal_type, ns=1000, nr=1000, at=20, cs=1):
    """Runs an annealing experiment and returns the sampleset
    
    :param bqm: binary quadratic model to be sampled
    :type bqm: BinaryQuadraticModel
    :param type: type of the annealing (simulated, hybrid, quantum)
    :type: string
    :param ns: Number of sweeps
    :type ns: int
    :param nr: Number of samples
    :type nr: int
    :param at: Annealing time
    :type at: int
    :param cs: Chain strength 
    :type cs: float
    :return: sampleset
    :rtype: dimod.SampleSet
    """
    if anneal_type == 's':
        s = neal.SimulatedAnnealingSampler()
        sampleset = s.sample(bqm, beta_range=(5, 100), num_sweeps=ns, num_reads=ns,
                             beta_schedule_type='geometric')
    elif anneal_type == 'h':
        s = LeapHybridSampler()
        sampleset = s.sample(bqm)
    elif anneal_type == 'q':
        s = EmbeddingComposite(DWaveSampler())
        sampleset = s.sample(bqm, chain_strength = cs, annealing_time = at, num_reads=1000)

    results, energies = [], []

    for datum in sampleset.data(fields=["sample", "energy"]):
        energies.append(datum.energy)
        results.append(datum[0])
    return sampleset

<b> Converting sample into a pitch sequence </b>

The output we get from D-Wave is not a pitch sequence but a list of binary variables and their values. After getting this output, we need to convert it into a pitch sequence.

In [52]:
def translate_sample(sample, P, n):
    """Converts a given sample into a pitch sequence
     
    :param sample: A dictionary representing a sample where keys are the variable names and values are either 0 or 1
    :param type sample: dict
    :param P: List of the pitches
    :param type P: list
    :param n: Number of notes in the sequence
    :param type n: int
    :return: sequence of pitches
    :rtype: list
    """
    seq = []
    for i,p in product(range(1,n+1),P):
        if sample[f"x_{i}_{p}"]==1:
            seq.append(p)
    return seq

<b> Converting sample into a chord sequence

In case we would like to create a chord sequence, there will be three notes at each time point. The following function creates a chord sequence from the sample.

In [53]:
from itertools import product

def translate_chord(sample,P,n):
    """Converts a given sample into a chord sequence
     
    :param sample: A dictionary representing a sample where keys are the variable names and values are either 0 or 1
    :param type sample: dict
    :param P: List of the pitches
    :param type P: list
    :param n: Number of notes in the sequence
    :param type n: int
    :return: sequence of chords
    :rtype: list
    """
    seq = {i:[] for i in range(1,n+1)}
    for i,p in product(range(1,n+1),P):
        if sample[f"x_{i}_{p}"]==1:
            seq[i].append(p)
    return seq

<b> Checking if constraints are broken using pyqubo

pyqubo has a built-in method to check whether the constraints defined using the `Constraint` keyword are broken or not. (It does not detect if there are penalty values resulting from the terms directly added to the objective.)

In [54]:
def check_constraints(sample, model):    
    """Prints the violated constraints in a given sample
     
    :param sample: A dictionary representing a sample where keys are the variable names and values are either 0 or 1
    :param type sample: dict
    :param model: Model that is representing the qubo problem
    :param type P: Pyqubo model
    """
    dec = model.decode_sample(sample, vartype='BINARY')
    print(dec.constraints(only_broken=True))

----

### Example 1 (Page 13) - Basic model for melody generation

<b> Creating the model

We first create a list of pitches `P` and set `n`, which is the number of notes in the music piece to be generated. 

In [55]:
P = ['C4', 'D4', 'E4','G4' ]
n = 5

Next we add the single note constraint, which is defined as 
$
\left ( 1- \sum_{j \in P} x_{i,j} \right )^2
$
for each $i=1,\dots,5$. (Eq. 18)

We use pyqubo to formulate the model. 

- $H$ will represent our QUBO formulation.
- Note that we have a for loop going through each $i$ and we add the constraint corresponding to $i$ to $H$.
- `Binary(x)` keyword defines a binary variable with name `x`
-  `sum(Binary(f"x_{i}_{j}") for j in P)` represents the sum $\sum_{j \in P} x_{i,j}$
- `Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")` adds the constraint $
\left ( 1- \sum_{j \in P} x_{i,j} \right )^2
$ to the model with the name `Single_note_{i}`


In [56]:
H = 0
for i in range(1,n+1):
    H += Constraint((1-sum(Binary(f"x_{i}_{j}") for j in P))**2, f"Single_note_{i}")

We can print `H` to see what it looks like.

In [57]:
H

(Constraint(((1.000000 + (-1.000000 * (Binary('x_5_G4') + Binary('x_5_E4') + Binary('x_5_D4') + 0.000000 + Binary('x_5_C4')))) * (1.000000 + (-1.000000 * (Binary('x_5_G4') + Binary('x_5_E4') + Binary('x_5_D4') + 0.000000 + Binary('x_5_C4'))))), 'Single_note_5') + Constraint(((1.000000 + (-1.000000 * (Binary('x_4_G4') + Binary('x_4_E4') + Binary('x_4_D4') + 0.000000 + Binary('x_4_C4')))) * (1.000000 + (-1.000000 * (Binary('x_4_G4') + Binary('x_4_E4') + Binary('x_4_D4') + 0.000000 + Binary('x_4_C4'))))), 'Single_note_4') + Constraint(((1.000000 + (-1.000000 * (Binary('x_3_G4') + Binary('x_3_E4') + Binary('x_3_D4') + 0.000000 + Binary('x_3_C4')))) * (1.000000 + (-1.000000 * (Binary('x_3_G4') + Binary('x_3_E4') + Binary('x_3_D4') + 0.000000 + Binary('x_3_C4'))))), 'Single_note_3') + Constraint(((1.000000 + (-1.000000 * (Binary('x_2_G4') + Binary('x_2_E4') + Binary('x_2_D4') + 0.000000 + Binary('x_2_C4')))) * (1.000000 + (-1.000000 * (Binary('x_2_G4') + Binary('x_2_E4') + Binary('x_2_D4') +

<b> Running the experiment

To run an annealing experiment using Ocean SDK, we need a BinaryQuadraticModel.

First we use pyqubo's `compile` function to generate the `model` object, which is then converted into a QUBO using `to_qubo()` function of pyqubo. 

From `qubo` and `of` representing the QUBO model and the offset respectively, we create a BinaryQuadraticModel object of D-Wave.

In [58]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

Let's print `bqm` as well.

In [59]:
bqm.shape

(20, 30)

The next step is to run the experiment. We use the `anneal` function we have previously written. In this example, we select quantum annealing by providing the parameter `q`. As we don't provide any additional parameters, default parameters will be used for annealing time and chain strength.

<i>Note: If you did not configure D-Wave access or you can not configure it due to your country of residence, then you can use 's' keyword to use simulated annealer instead. 

In [66]:
sampleset = anneal(bqm, 's')

<b> Interpreting the results

Let's see what samples look like. 

In [67]:
sampleset

SampleSet(rec.array([([0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], 0., 1),
           ([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0], 0., 1),
           ([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1], 0., 1),
           ([1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0], 0., 1),
           ([0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], 0., 1),
           ([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], 0., 1),
           ([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1], 0., 1),
           ([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0], 0., 1),
           ([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1], 0., 1),
           ([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0], 0., 1),
           ([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0], 0., 1),
           ([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], 0., 1),
      

When you consider a line from the samples list, the list is the value of the binary variables, 0 indicates that the energy is 0, and 1 indicates that this specific sample is observed only once.
`[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], 0., 1)`

The order of the variables can be seen from the list of variables.

`Variables(['x_1_C4', 'x_1_D4', 'x_1_E4', 'x_1_G4', 'x_2_C4', 'x_2_D4', 'x_2_E4', 'x_2_G4', 'x_3_C4', 'x_3_D4', 'x_3_E4', 'x_3_G4', 'x_4_C4', 'x_4_D4', 'x_4_E4', 'x_4_G4', 'x_5_C4', 'x_5_D4', 'x_5_E4', 'x_5_G4']`

Finally, let's print the corresponding note sequences to the sampleset we have obtained. We take the first 5 samples.

In [68]:
for sample in sampleset.samples()[:5]:
    seq = translate_sample(sample,P,n)
    print(seq)

['C4', 'G4', 'D4', 'E4', 'E4']
['E4', 'E4', 'E4', 'D4', 'G4']
['G4', 'G4', 'G4', 'E4', 'D4']
['G4', 'D4', 'G4', 'E4', 'D4']
['C4', 'C4', 'D4', 'E4', 'D4']


In [69]:
midi.writeToMidi(seq,"seq.mid")
midi.playMidi("seq.mid")

We can also print all samples and their energies.

In [17]:
for data in sampleset.data():
    seq = translate_sample(data.sample,P,n)
    print(seq,data.energy)

['E4', 'C4', 'G4', 'C4', 'C4'] 0.0
['C4', 'G4', 'C4', 'G4', 'G4'] 0.0
['C4', 'E4', 'G4', 'E4', 'C4'] 0.0
['E4', 'E4', 'C4', 'G4', 'C4'] 0.0
['G4', 'D4', 'G4', 'D4', 'G4'] 0.0
['G4', 'E4', 'C4', 'D4', 'C4'] 0.0
['G4', 'E4', 'D4', 'D4', 'C4'] 0.0
['D4', 'C4', 'D4', 'G4', 'E4'] 0.0
['D4', 'G4', 'G4', 'E4', 'G4'] 0.0
['E4', 'C4', 'D4', 'D4', 'G4'] 0.0
['G4', 'D4', 'C4', 'C4', 'E4'] 0.0
['E4', 'D4', 'C4', 'C4', 'G4'] 0.0
['G4', 'G4', 'G4', 'G4', 'D4'] 0.0
['D4', 'E4', 'D4', 'G4', 'C4'] 0.0
['E4', 'C4', 'G4', 'D4', 'G4'] 0.0
['E4', 'G4', 'E4', 'E4', 'G4'] 0.0
['E4', 'C4', 'C4', 'G4', 'G4'] 0.0
['C4', 'E4', 'C4', 'E4', 'G4'] 0.0
['G4', 'C4', 'G4', 'G4', 'G4'] 0.0
['G4', 'G4', 'G4', 'D4', 'D4'] 0.0
['C4', 'G4', 'D4', 'C4', 'G4'] 0.0
['D4', 'C4', 'E4', 'E4', 'C4'] 0.0
['C4', 'E4', 'G4', 'G4', 'E4'] 0.0
['C4', 'C4', 'E4', 'E4', 'E4'] 0.0
['D4', 'G4', 'G4', 'G4', 'G4'] 0.0
['G4', 'C4', 'D4', 'E4', 'E4'] 0.0
['D4', 'D4', 'D4', 'C4', 'C4'] 0.0
['D4', 'E4', 'E4', 'E4', 'E4'] 0.0
['D4', 'D4', 'G4', '

You can get the sample with the lowest energy using `sampleset.first.sample`. Since there are multiple of them in this case, this will return you the first sample in the list.

In [18]:
sampleset.first.sample

{'x_1_C4': np.int8(0),
 'x_1_D4': np.int8(0),
 'x_1_E4': np.int8(1),
 'x_1_G4': np.int8(0),
 'x_2_C4': np.int8(1),
 'x_2_D4': np.int8(0),
 'x_2_E4': np.int8(0),
 'x_2_G4': np.int8(0),
 'x_3_C4': np.int8(0),
 'x_3_D4': np.int8(0),
 'x_3_E4': np.int8(0),
 'x_3_G4': np.int8(1),
 'x_4_C4': np.int8(1),
 'x_4_D4': np.int8(0),
 'x_4_E4': np.int8(0),
 'x_4_G4': np.int8(0),
 'x_5_C4': np.int8(1),
 'x_5_D4': np.int8(0),
 'x_5_E4': np.int8(0),
 'x_5_G4': np.int8(0)}

As the lowest energy sample has energy 0, we expect that no constraint is violated. We can check it also using pyqubo. The returned dictionary will contain the list of violated constraints, or {} in case no constraint is violated. 

In [19]:
check_constraints(sampleset.first.sample, model)  

{}


If you want to store the results, you can run the following cell. 

In [20]:
filename ="experiment1"
#store_results(filename, sampleset)

---

### Example 2 (page 14) - Avoiding certain consecutive notes

<b> Creating the model

We use the same `P` and `n` as before.

In [21]:
P = ['C4', 'D4', 'E4','G4' ]
n = 5

First we define the single note constraint. (Eq. 18)

In [22]:
H = 0
for i in range(1,n+1):
    H += Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We want to enforce that G4 does not follow D4. We add the term ` Binary(f"x_{i}_D4")*Binary(f"x_{i+1}_G4")` directly to the objective for each $i$. Note that the penalty coefficient is 1 in this case. (Eq.20)

In [23]:
for i in range(1,n+1):
    H += Binary(f"x_{i}_D4")*Binary(f"x_{i+1}_G4")     

We will force that the same note does not appear more than twice in a row. (Eq. 22)

$ x_{i,j} + x_{i+1,j} + x_{i+2,j} \leq 2$

As we have an inequality, we need to introduce a slack variable $s_{i,j}$. 

$ x_{i,j} + x_{i+1,j} + x_{i+2,j} +s_{i,j} = 2$

The upper bound for the slack variable is 2, as all three binary variables can be equal to 0 simulatenously. Hence we need an integer variable $s$. We use `LogEncInteger(f"s_{i}_{j}",(0,2))` from pyqubo to create an integer variable named `s_i_j`, with the lower and upper bounds (0,2). 

Finally, we convert it to the following constraint add to our model.

$ (x_{i,j} + x_{i+1,j} + x_{i+2,j} +s_{i,j} - 2)^2$

Notice that the penalty value is set as 1.

In [24]:
for i in range(1,n-1):
    for j in P:
        H += Constraint((Binary(f"x_{i+2}_{j}") + Binary(f"x_{i}_{j}") + Binary(f"x_{i+1}_{j}") + LogEncInteger(f"s_{i}_{j}",(0,2))-2)**2, f"same_note_{i}_{j}")

<b> Running the experiment

In [25]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

We increase the chain strength to 1.5 this time.

In [26]:
sampleset = anneal(bqm, 's', cs=1.5)

<b>Interpreting the results

Let's print the first 5 samples and their energies.

In [27]:
for i, data in enumerate(sampleset.data()):
    seq = translate_sample(data.sample,P,n)
    print(seq,data.energy)
    if i==5:
        break

['G4', 'C4', 'C4', 'G4', 'D4'] 0.0
['D4', 'E4', 'G4', 'D4', 'C4'] 0.0
['C4', 'G4', 'E4', 'C4', 'G4'] 0.0
['D4', 'E4', 'G4', 'C4', 'E4'] 0.0
['C4', 'E4', 'E4', 'D4', 'D4'] 0.0
['E4', 'E4', 'D4', 'C4', 'G4'] 0.0


In [28]:
filename = "experiment2"
store_results(filename, sampleset)

---

### Example 3 (Page 14) - Semitones and avoiding augmented intervals

 <b> Creating the model

We select the subset of the chromatic scale as `P` and we will generate 20 notes.

In [29]:
P= [0,2,4,5,7,9,11,12]
n = 20

`A` is the list of unallowed intervals.

In [30]:
A= [6,8,10,12]

This time, we will set some penalty values to be  different from 1.

In [31]:
P1, P2, P3, P4 = 20,2,1,1

We start with the single note constraint. (Eq.18)

In [32]:
H = 0
for i in range(1,n+1):
    H += Constraint(P1*(1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We include the rule about no more than two same note in a sequence. (Eq.22)

In [33]:
for i in range(1,n-1):
    for j in P:
        H += Constraint(P2*(Binary(f"x_{i+2}_{j}") + Binary(f"x_{i}_{j}") + Binary(f"x_{i+1}_{j}") + LogEncInteger(f"s_{i}_{j}",(0,2))-2)**2, f"same_note_{i}_{j}")

We add penalty terms to the objective for the unallowed intervals, by adding the following term: $   x_{i,j}x_{i+1,j'}  $
for all $i \in [n], |j'-j| \in A$. (Eq.24) 



In [34]:
for i in range(1,n):
    for j, k in product(P,P):
        if abs(j-k) in A:
            H += P3*Binary(f"x_{i}_{j}")*(Binary(f"x_{i+1}_{k}"))

We add the rule about the first and the last notes by adding the following terms to the objective: $(1-x_{1,0}),~~(1-x_{n,0})$. (Eq.26)

In [35]:
H += P4*(1-Binary(f"x_1_0")) + P4*(1-Binary(f"x_20_0"))   

<b> Running the experiment

In [36]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

This time, we will use the hybrid solver.

In [37]:
sampleset = anneal(bqm, 's')

Hybrid solver returns only a single solution.

In [38]:
for sample in sampleset.samples()[:1]:
    seq = translate_sample(sample,P,n)
    print(seq)

[0, 4, 9, 7, 12, 9, 5, 9, 9, 7, 9, 9, 7, 9, 9, 7, 7, 5, 4, 9]


The following code provided by pyqubo can be used to identify the constraints that are broken (if any). You can try running the same experiment by trying quantum annealing directly and check if some constraints are violated or not. What you can try when some constraint is violated is to increase its penalty value, and overall increase the annealing time and number of reads. Still, you may not be able to get a sample in which all constraints are satisfied. In such a case, you can run hybrid solver.

In [39]:
dec = model.decode_sample(sampleset.first.sample, vartype='BINARY')
print(dec.constraints(only_broken=True))

{}


In [40]:
filename = "experiment4"
store_results(filename, sampleset)

---

### Example 4 (Page 15) - Diatonic scale and tendency notes

<b>Creating the model

This time, we take the diatonic scale and select `P` as follows. We will generate 20 notes.

In [72]:
P = [f'{x}4' for x in string.ascii_uppercase[:7]]
n = 20

We start by adding the single note constraint. (Eq. 18)

In [73]:
H = 0
for i in range(1,n+1):
    H += Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We initialize the first and the last note of the sequence. (Eq. 26)

In [74]:
H += (1-Binary(f"x_1_A4")) + (1-Binary(f"x_20_A4"))

Next we add the rules about tendency notes (Eq.28), by adding the following terms directly to the objective.

In [75]:
for i in range(1,n):
    H += Binary(f"x_{i}_2")*(1-Binary(f"x_{i+1}_1"))
    H += Binary(f"x_{i}_4")*(1-Binary(f"x_{i+1}_3"))
    H += Binary(f"x_{i}_6")*(1-Binary(f"x_{i+1}_5"))
    H += Binary(f"x_{i}_7")*(1-Binary(f"x_{i+1}_8"))

<b> Running the experiment

In [76]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)
bqm.shape

(292, 496)

In [77]:
cs = 1.5
sampleset = anneal(bqm, 's')

<b> Interpreting the results

Let's print the first five samples and their energies.

In [78]:
for i,data in enumerate(sampleset.data()):
    seq = translate_sample(data.sample,P,n)
    print(seq,data.energy)
    if i==6:
        break

midi.writeToMidi(seq, "seq2")
midi.playMidi("seq2")

['A4', 'G4', 'A4', 'C4', 'D4', 'A4', 'C4', 'C4', 'G4', 'D4', 'E4', 'A4', 'B4', 'D4', 'C4', 'E4', 'D4', 'G4', 'D4', 'A4'] 0.0
['A4', 'A4', 'A4', 'F4', 'A4', 'G4', 'B4', 'B4', 'G4', 'E4', 'B4', 'F4', 'D4', 'F4', 'D4', 'B4', 'A4', 'A4', 'E4', 'A4'] 0.0
['A4', 'D4', 'G4', 'B4', 'B4', 'B4', 'F4', 'B4', 'B4', 'F4', 'D4', 'G4', 'G4', 'F4', 'A4', 'E4', 'C4', 'F4', 'A4', 'A4'] 0.0
['A4', 'E4', 'G4', 'G4', 'B4', 'D4', 'E4', 'F4', 'E4', 'F4', 'G4', 'D4', 'C4', 'E4', 'D4', 'D4', 'G4', 'F4', 'D4', 'A4'] 0.0
['A4', 'F4', 'C4', 'G4', 'B4', 'G4', 'A4', 'F4', 'E4', 'B4', 'F4', 'B4', 'B4', 'D4', 'E4', 'F4', 'G4', 'F4', 'D4', 'A4'] 0.0
['A4', 'E4', 'B4', 'G4', 'A4', 'B4', 'F4', 'E4', 'E4', 'B4', 'B4', 'C4', 'C4', 'C4', 'D4', 'F4', 'D4', 'B4', 'G4', 'A4'] 0.0
['A4', 'G4', 'C4', 'G4', 'F4', 'G4', 'F4', 'C4', 'C4', 'B4', 'A4', 'A4', 'G4', 'G4', 'B4', 'B4', 'A4', 'B4', 'B4', 'A4'] 0.0


In [ ]:
filename = "experiment4"
store_results(filename, sampleset)

---

### Example 5 (Page 16-17) - Objective function

<b>Creating the model

We use the diatonic scale as the list of pitches and we will generate 20 notes.

In [273]:
P = [f'{x}4' for x in string.ascii_uppercase[:7]]
n = 20

We define some penalty values. 

In [274]:
P1,P2,P3,P4,P5 = 5,3,1,1,1

We define the matrix of the weights.

In [275]:
M = [[0 for i in range(9)] for j in range(9)]
M[4][4]=2
M[4][3]=2
M[4][5]=1

M[5][6]=1
M[5][4]=1

M[6][6]=1
M[6][5]=1


M[2][1]=1
M[2][3]=1
M[2][2]=1

M[1][1]=1
M[1][2]=1

As always, we start by single note constraint. (Eq.18)

In [276]:
H = 0
for i in range(1,n+1):
    H += P1*Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We add the rule about the first and the last note. (Eq.26)

In [277]:
#H += P2*Constraint(1-Binary(f"x_1_1"),"first") + P2*Constraint(1-Binary(f"x_20_1"),"last")

We include the rule about no more than two same note in a sequence. (Eq.22)

In [278]:
for i in range(1,n-1):
    for j in P:
        H += Constraint(P3*(Binary(f"x_{i+2}_{j}") + Binary(f"x_{i}_{j}") + Binary(f"x_{i+1}_{j}") + LogEncInteger(f"s_{i}_{j}",(0,2))-2)**2, f"same_note_{i}_{j}")

Next we add the rules about tendency notes (Eq.28), by adding the following terms directly to the objective.

In [279]:
'''
for i in range(1,n):
    H += P4*Binary(f"x_{i}_2")*(1-Binary(f"x_{i+1}_1"))
    H += P4*Binary(f"x_{i}_4")*(1-Binary(f"x_{i+1}_3"))
    H += P4*Binary(f"x_{i}_6")*(1-Binary(f"x_{i+1}_5"))
    H += P4*Binary(f"x_{i}_7")*(1-Binary(f"x_{i+1}_8"))
'''

'\nfor i in range(1,n):\n    H += P4*Binary(f"x_{i}_2")*(1-Binary(f"x_{i+1}_1"))\n    H += P4*Binary(f"x_{i}_4")*(1-Binary(f"x_{i+1}_3"))\n    H += P4*Binary(f"x_{i}_6")*(1-Binary(f"x_{i+1}_5"))\n    H += P4*Binary(f"x_{i}_7")*(1-Binary(f"x_{i+1}_8"))\n'

Finally, we add the 'rewards' to our objective function, defined as $
    -\sum_{ \substack{i\in [n-1]  \\ j,j' \in P}} W_{j,j'} x_{i,j} x_{i+1,j'}
$. The weights we use are stored inside `M`. (Eq. 29)



In [280]:
for i in range(1,n+1):
    for j, jp in product(P,P):
        if M[P.index(j)][P.index(jp)]!=0:
            H += -P5*M[P.index(j)][P.index(jp)]*Binary(f"x_{i}_{j}")*Binary(f"x_{i+1}_{jp}")           

<b> Running the experiment

In [281]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

We'll again use the hybrid solver.

In [282]:
sampleset = anneal(bqm, 's')

<b> Interpreting the results

In [283]:
for data in sampleset.data():
    seq = translate_sample(data.sample,P,n)
    print(seq,data.energy)

midi.writeToMidi(seq,"mat")
midi.playMidi("mat")

['F4', 'G4', 'E4', 'E4', 'F4', 'G4', 'G4', 'F4', 'G4', 'F4', 'G4', 'G4', 'F4', 'E4', 'E4', 'F4', 'G4', 'G4', 'E4', 'E4'] -25.0
['G4', 'G4', 'F4', 'G4', 'E4', 'E4', 'F4', 'G4', 'F4', 'E4', 'D4', 'G4', 'F4', 'G4', 'E4', 'E4', 'F4', 'G4', 'E4', 'E4'] -24.0
['G4', 'G4', 'C4', 'E4', 'E4', 'F4', 'E4', 'E4', 'D4', 'F4', 'G4', 'G4', 'F4', 'E4', 'E4', 'D4', 'G4', 'E4', 'D4', 'E4'] -24.0
['G4', 'F4', 'G4', 'G4', 'F4', 'E4', 'E4', 'F4', 'G4', 'G4', 'F4', 'G4', 'C4', 'D4', 'E4', 'F4', 'E4', 'F4', 'E4', 'E4'] -24.0
['E4', 'E4', 'D4', 'C4', 'B4', 'E4', 'F4', 'G4', 'F4', 'E4', 'G4', 'F4', 'E4', 'F4', 'G4', 'F4', 'E4', 'E4', 'F4', 'E4'] -24.0
['G4', 'F4', 'E4', 'F4', 'G4', 'F4', 'G4', 'E4', 'E4', 'F4', 'G4', 'G4', 'E4', 'F4', 'E4', 'F4', 'G4', 'E4', 'E4', 'E4'] -23.0
['F4', 'E4', 'E4', 'F4', 'D4', 'F4', 'G4', 'G4', 'F4', 'E4', 'D4', 'E4', 'E4', 'F4', 'E4', 'F4', 'G4', 'F4', 'G4', 'E4'] -23.0
['E4', 'D4', 'G4', 'G4', 'E4', 'G4', 'F4', 'F4', 'E4', 'E4', 'F4', 'E4', 'D4', 'G4', 'G4', 'F4', 'G4', 'F4', 'E

In [284]:
dec = model.decode_sample(sampleset.first.sample, vartype='BINARY')
print(dec.constraints(only_broken=True))

{}


In [285]:
filename = "experiment5"
store_results(filename, sampleset)

---

### Example 6 (Page 17-18) - Rhythm generation

<b> Creating the model

We identify quarter, dotted quarted, eighth and half notes by 0,1,2,3 respectively. Later on, we will use this `Pdict` to translate our sample back.

In [286]:
Pdict = {0:"Q", 1:"DQ", 2:"E", 3:"H"}
P = [0,1,2,3]
n = 20

We define the matrix of weights for the note durations.

In [287]:
M = [[0 for i in range(4)] for j in range(4)]
M[0][0]=11
M[0][1]=1

M[1][2] = 1
M[2][3]=1

Penalty values.

In [288]:
P1, P2, P3 = 50, 50, 1

We start by single note constraint. (Eq.18)

In [289]:
H = 0
for i in range(1,n+1):
    H += P1*Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We add the rule forcing that each duration appears at least twice. (Eq.36)

In [290]:
for d in P:
    H += P2*Constraint((2-sum(Binary(f"x_{i}_{d}") for i in range(1,n+1))+LogEncInteger(f"s_{d}",(0,14)))**2, f"At_least_two_{d}")

We add the 'rewards' reflecting the weights for consecutive durations. (Eq. 29)

In [291]:
for i in range(1,n+1):
    for j, jp in product(P,P):
        if M[j][jp]!=0:
            H += P3 * -M[j][jp]*Binary(f"x_{i}_{j}")*Binary(f"x_{i+1}_{jp}")

<b> Running the experiment

In [292]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

We will use the hybrid solver

In [294]:
sampleset = anneal(bqm, 's')

<b> Interpreting the results

In [295]:
for data in sampleset.data():
    seq = translate_sample(data.sample,P,n)
    print(seq,data.energy)

[2, 3, 0, 1, 3, 3, 0, 1, 1, 1, 3, 3, 1, 3, 3, 3, 2, 0, 3, 0, 1] 33.0
[1, 2, 0, 1, 2, 3, 2, 3, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 2, 1, 2, 2, 0] 72.0
[0, 0, 0, 0, 2, 3, 2, 2, 3, 0, 3, 1, 1, 2, 3, 2, 0, 3, 0, 0, 2, 3, 0] 75.0
[1, 2, 0, 3, 0, 2, 3, 1, 2, 3, 0, 2, 0, 3, 0, 0, 0, 0, 2, 2, 3, 0, 3] 77.0
[3, 3, 0, 1, 3, 0, 1, 3, 0, 0, 0, 0, 1, 0, 1, 0, 1, 3, 2, 3, 0, 1, 2] 85.0
[0, 0, 1, 0, 0, 2, 3, 3, 0, 0, 2, 3, 0, 2, 3, 3, 2, 2, 0, 3, 0, 1, 2] 87.0
[0, 0, 0, 3, 2, 3, 1, 2, 2, 3, 2, 3, 3, 3, 3, 1, 0, 0, 0, 2, 3, 2, 3] 99.0
[3, 2, 1, 2, 3, 2, 3, 2, 2, 3, 0, 3, 0, 0, 1, 2, 0, 3, 0, 3, 3, 0, 1] 107.0
[0, 0, 0, 0, 0, 0, 2, 1, 3, 2, 3, 1, 2, 3, 1, 2, 1, 3, 2, 1, 3, 2, 3, 1] 132.0
[3, 0, 0, 0, 1, 1, 1, 2, 2, 3, 0, 1, 2, 1, 3, 2, 3, 1, 2, 3, 2, 0, 3, 0] 142.0
[2, 0, 3, 0, 1, 0, 1, 2, 3, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 1, 2, 0, 3] 151.0
[0, 1, 2, 0, 3, 1, 1, 2, 3, 1, 2, 0, 3, 0, 1, 2, 3, 2, 3, 1, 2, 0, 3, 0] 152.0
[2, 3, 1, 2, 0, 3, 0, 1, 2, 3, 1, 2, 3, 0, 1, 2, 2, 3, 3, 0, 1, 0, 0, 1] 163.0
[0, 1, 2, 

We can get the list of durations back using the dictionary.

In [296]:
L = [Pdict[s] for s in seq]
print(L)

['DQ', 'E', 'Q', 'H', 'DQ', 'E', 'E', 'H', 'E', 'H', 'Q', 'DQ', 'H', 'Q', 'E', 'Q', 'H', 'Q', 'H', 'Q', 'E', 'Q', 'H', 'Q', 'E', 'E', 'H', 'E', 'H', 'E', 'H', 'Q', 'H', 'Q', 'E', 'DQ', 'H', 'Q', 'E', 'Q', 'DQ']


In [297]:
dec = model.decode_sample(sampleset.first.sample, vartype='BINARY')
print(dec.constraints(only_broken=True))

{'Single_note_18': (False, 1.0)}


In [298]:
filename = f"experiment6"
store_results(filename, sampleset)

---

### Example 7 (Page 19-20-21) Markov Random Fields

<b> Creating the model</b>

In order to build a Markov Network it is neccessary to create a dictionary which will contain the information about the potentials relating each pair of binary random variables. We want to have a dictionary with the following format:

```
potentials = {('a', 'b'): {(0, 0): 1,  # for when a = 0, b = 0
                          (0, 1): .5,  # for when a = 0, b = 1
                          (1, 0): .5,  # for when a = 1, b = 0
                          (1, 1): 2}}  # for when a = 1, b = 1
```

For our harmonization task we are going to create a list with the chords that we are going to be considering.

We are also going to assign values to all the combinations of two chords that we can have in our sequence. These values will be the values for the potentials.

Let's start by defiining the `chord_list`.

In [ ]:
chords_list= ['I','ii','iii','IV','V','vi','viid']

We'll create a dictionary named potentials, whose keys will pairs of variables representing the chord at each time point, i.e. $\mathtt{I}_i, \mathtt{ii}_i, \mathtt{iii}_i, \mathtt{IV}_i, \mathtt{V}_i, \mathtt{vi}_i, \mathtt{VIIdim}_i$ where $i\in [4]$

In [ ]:
potentials = {} 

We will use the following values to avoid having multiple chords at a time point. (Table 7)

In [ ]:
not_simultaneous = {(0, 0): 50,(0, 1): 50,(1, 0): 50,(1, 1): 100} # to avoid having two simultaneous chords

We define the potentials to be used when both variables are `(1,1)`. (Table 8 and Table 9)

In [ ]:
values={('I','I'):100, ('I','ii'):50,('I','iii'):50,('I','IV'):50, ('I','V'):50, ('I','vi'):50, ('I','viid'):50,
             ('ii','I'):50, ('ii','ii'):100,('ii','iii'):50,('ii','IV'): 50, ('ii','V'): 50, ('ii','vi'):50, ('ii','viid'):50,
             ('iii','I'):50, ('iii','ii'):50,('iii','iii'):100,('iii','IV'):50, ('iii','V'):50, ('iii','vi'):50, ('iii','viid'):50,
             ('IV','I'):50, ('IV','ii'):50,('IV','iii'):50,('IV','IV'):100, ('IV','V'):50, ('IV','vi'):50, ('IV','viid'):50,
             ('V','I'):0, ('V','ii'):50,('V','iii'):50,('V','IV'):50, ('V','V'):50, ('V','vi'):50, ('V','viid'):50,
             ('vi','I'):50, ('vi','ii'):50,('vi','iii'):50,('vi','IV'):50, ('vi','V'):50, ('vi','vi'):100, ('vi','viid'):50,
             ('viid','I'):50, ('viid','ii'):50,('viid','iii'):50,('viid','IV'):50, ('viid','V'):50, ('viid','vi'):50, ('viid','viid'):100
            }

Finally we fill the potentials dictionary. Note that we need to consider combinations of all time points and chords.

In [ ]:
for i, j in product(range(1,4),chords_list): # in this example we consider a sequence of four chords only
    for m in chords_list:
        index = chords_list.index(m)
        inner_list = chords_list[index+1:(len(chords_list))]
        for n in inner_list:
            potentials[(f"{i}_{m}",f"{i}_{n}")]= not_simultaneous # to avoid simultaneous chords
    for k in chords_list:
            potentials[(f"{i}_{j}",f"{i+1}_{k}")]={(0, 0): 50,(0, 1): 50,(1, 0): 50,(1, 1): values[(j,k)]}

In the previous cell we have considered the last chord as i was restricted to 1,2,3. We avoid having muliple chords as the last chord using the following cell. 

In [ ]:
last_chord = 4
for j in chords_list:
    index = chords_list.index(j)
    inner_list = chords_list[index+1:(len(chords_list))]
    for k in inner_list:
        potentials[(f"{last_chord}_{j}",f"{last_chord}_{k}")] = not_simultaneous # to avoid simultaneous chords

<b>Running the experiment

We first create a markov network using the networx. 

In [ ]:
MN = dnx.markov_network(potentials) # create a Markov Network

Next, we create the sample and use `sample_markov_network` function. We can use the `fixed_variables` parameter to set certain variables. In this example, we fix the first and the last chords. 

In [ ]:
sampler = LeapHybridSampler()
sample = dnx.sample_markov_network(MN,sampler, fixed_variables={
     '1_I': 1, '4_I': 1}) # let's consider a 4 chords progression starting in I and ending in I

<b> Interpreting the results

Let's check how the sample looks like. At a certain time point, there should be exactly one variable which is set to 1.

In [ ]:
sample[0]

The variables that are set to 1 will give us the chord sequence we are looking for. 

In [ ]:
chord_sequence = [k for k,v in sample[0].items() if v==1]
print(chord_sequence)

`sample` is a list containing only a single sample and we can directly store it using pickle.

In [ ]:
filename = f"experiment7"
with open(filename, 'wb') as handle:
    pickle.dump(sample, handle)

---

### Example 8 (Page 22) - Harmonizing melody

<b> Creating the model

The list `P` represents the degrees of the scale

In [ ]:
P = [1,2,3,4,5,6,7,8]
n = 20

The `note_list` represents the music pieace we would like to harmonize.

In [ ]:
note_list = [1, 4, 3, 5, 1, 7, 8, 3, 5, 4, 3, 5, 4, 3, 6, 5, 4, 3, 5, 1]

We would like to generate a triad. Hence, as opposed to single note constraint, we have the constraint forcing that there are three notes at each time point $
\sum_{j \in P} x_{i,j} = 3
$. (Eq.41)



In [ ]:
H = 0
for i in range(1,n+1):
    H += 2*Constraint((3-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Triad_{i}")

We would like to have the first and the last triad to be built on the first degree of the scale, hence we use the following constraints. (Eq. 42)

In [ ]:
H += Constraint((1-Binary(f"x_{1}_{3}"))**2, f"note_{1}_3")
H += Constraint((1-Binary(f"x_{1}_{5}"))**2, f"note_{1}_5")
H += Constraint((1-Binary(f"x_{20}_{3}"))**2, f"note_{20}_3")
H += Constraint((1-Binary(f"x_{20}_{5}"))**2, f"note_{20}_5")
H += Constraint((1-Binary(f"x_{20}_{1}"))**2, f"note_{20}_1")

Each triad should contain the note given in the `note_list`.

In [ ]:
for i in range(1,n+1):
    H += Constraint((1-Binary(f"x_{i}_{note_list[(i-1)]}"))**2, f"note_{i}")

We would like to avoid consecutive degrees to appear in the triad, as well as degrees 1-7, 2-6, 1-8. Hence we add the following penalty terms to the objective. (Eq. 44)

In [ ]:
A = [1,6,7]
for i in range(1,n+1):
    for j, k in product(P,P):
        if abs(j-k) in A:
            H += Binary(f"x_{i}_{j}")*(Binary(f"x_{i}_{k}"))

<b> Running the experiment

In [ ]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

In [ ]:
sampleset = anneal(bqm, 'h')

<b> Interpreting the results

We use the method `translate_chord` to convert our sample into a list of chords.

In [ ]:
for data in sampleset.data():
    seq = translate_chord(data.sample,P,n)
    print(seq,data.energy)

You can check if any of the constraints are broken or not.

In [ ]:
dec = model.decode_sample(sampleset.first.sample, vartype='BINARY')
print(dec.constraints(only_broken=True))

In [ ]:
filename = f"experiment8"
store_results(filename, sampleset)